In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

features1 = pd.read_csv('/Users/vaibhavrangan/Downloads/Stat_303-1/Data/House_features1.csv')
features2 = pd.read_csv('/Users/vaibhavrangan/Downloads/Stat_303-1/Data/House_features2.csv')
all_prices = pd.read_csv('/Users/vaibhavrangan/Downloads/Stat_303-1/Data/House_all_prices.csv')
prices = pd.read_csv('/Users/vaibhavrangan/Downloads/Stat_303-1/Data/House_prices.csv')

Q26

In [2]:
combined_features = pd.concat([features1, features2])
print(len(combined_features))

7300


Q27

In [3]:
combined_features_pivot = combined_features.pivot(index = 'House_Id', columns = 'variable')
print(combined_features_pivot.shape[1])

5


Q28

In [4]:
combined_features_pivot.columns = combined_features_pivot.columns.droplevel(0)
# Reset the index so that House_Id becomes a column
combined_features_pivot.reset_index(inplace = True)
# merge with prices
merged = pd.merge(combined_features_pivot, prices)
print(len(merged))

1460


Q29

In [5]:
# group by Neighborhood and apply std function on SalePrice
variance = merged.groupby('Neighborhood')["SalePrice"].std().sort_values(ascending = False)
variance.head()

Neighborhood
NoRidge    116120.661615
StoneBr    114583.197743
NridgHt     99248.411050
Crawfor     69321.858118
Timber      67128.253482
Name: SalePrice, dtype: float64

Q30

In [6]:
merged["GrLivArea"] = merged["GrLivArea"].astype(int)
filtered = merged.groupby("Neighborhood").filter(lambda x: len(x) > 2)
corr_by_hood = filtered.groupby("Neighborhood").apply(
        lambda x: x.corr().loc["GrLivArea", "SalePrice"]
    ).sort_values(ascending = False)

corr_by_hood.head()

Neighborhood
NoRidge    0.885033
CollgCr    0.865722
BrkSide    0.844037
StoneBr    0.842635
SawyerW    0.810779
dtype: float64

Q31

In [7]:
row_count = merged.shape[0]
# inclusive of left bound exclusive of right bound
# bins are [0,4), [4,5), [5,6), [6,7), [7,10)
merged["Binned_Qual"] = pd.cut(merged["OverallQual"].astype(int), bins = (0,4, 5,6,7,10), labels = ("1", "2", "3", "4", "5"))
print(100*merged["Binned_Qual"].value_counts()/row_count)

2    27.191781
3    25.616438
4    21.849315
5    15.684932
1     9.657534
Name: Binned_Qual, dtype: float64


Q32

In [8]:
merged_imputed = merged.copy()
na_count = merged_imputed["SalePrice"].isnull().sum()
grouped = merged.groupby(["BldgType", "Binned_Qual"], group_keys=False, observed=True)
merged_imputed["SalePrice"] = grouped["SalePrice"].apply(lambda x: x.fillna(x.mean()))

# take mean of rmse across imputed values, not all values otherwise will skew to a lot of 0s because error is 0 for all non imputed values
rmse = np.sqrt(((all_prices['SalePrice'] - merged_imputed['SalePrice'])**2).sum()/na_count)
print(rmse)

51410.62720668922


Q33

In [9]:
def trendline_impute(df):  
    y = df.loc[~df['SalePrice'].isnull(),'SalePrice']
    x = df.loc[~df['SalePrice'].isnull(),'GrLivArea']    
    z = np.polyfit(x,y,1)
    p = np.poly1d(z)
    xpred = df.loc[df['SalePrice'].isnull(),'GrLivArea']
    df.loc[df['SalePrice'].isnull(),'SalePrice'] = p(xpred)
    return(df)

trendline_impute_data = merged.copy()
# since the function specifies which column to apply the imputation to and returns a df, can set it to trendline_impute_data
trendline_impute_data = trendline_impute(trendline_impute_data)
rmse = np.sqrt(((all_prices['SalePrice'] - trendline_impute_data['SalePrice'])**2).sum()/na_count)
print(rmse)

57453.59130006382


Q34

In [10]:
data_imputed = merged.copy()
total_imputed = data_imputed["SalePrice"].isnull().sum()
data_imputed["SalePrice"] = data_imputed.groupby(["BldgType", "Binned_Qual"], group_keys= False, observed=True).apply(trendline_impute)
rmse = np.sqrt(((all_prices['SalePrice'] - data_imputed['SalePrice'])**2).sum()/total_imputed)
print(rmse)    

635916.6362570759


Q35

In [ ]:
grouped = merged.groupby(["BldgType", "Binned_Qual"], group_keys=False, observed=True)
def rmse():
    return np.sqrt(((all_prices['SalePrice'] - data_imputed['SalePrice'])**2).sum()/merged.SalePrice.isnull().sum())

def price_j(x, j):
    idx_missing = x.index[x["SalePrice"].isnull()]
    for i in idx_missing:
        dist = (abs(x.loc[i, "GrLivArea"] - x.loc[~x["SalePrice"].isnull(), "GrLivArea"])).sort_values().head(j).index
        x.loc[i, "SalePrice"] = x.loc[dist, "SalePrice"].mean()
    return(x)

for j in range(1,21):
    data_imputed = grouped.apply(lambda x: price_j(x, j))
    print("j = ", j, "RMSE = ", rmse())



j =  1 RMSE =  51816.18799984918
j =  2 RMSE =  46072.66568836822
j =  3 RMSE =  44386.70600388442
j =  4 RMSE =  40528.984238960926
j =  5 RMSE =  39605.71573952534
j =  6 RMSE =  38788.01545039912
j =  7 RMSE =  39219.09755973458
j =  8 RMSE =  40857.37603408943
j =  9 RMSE =  39541.38064611444
j =  10 RMSE =  38576.62546979164
j =  11 RMSE =  39047.92526628425
j =  12 RMSE =  39097.701333020814
j =  13 RMSE =  39039.8341272041
j =  14 RMSE =  39117.712904742875
j =  15 RMSE =  38592.752662710074
j =  16 RMSE =  38871.5207958454
j =  17 RMSE =  38984.04841138605
j =  18 RMSE =  38784.409889022434
j =  19 RMSE =  38773.60401990701
j =  20 RMSE =  38926.08556613929
j =  10 RMSE =  38576.62546979164
